In [ ]:
import openpyxl
from google.colab import files

def determine_semester(semester_text):
    if '1학기' in semester_text:
        return 1
    elif '2학기' in semester_text:
        return 2
    elif '여름' in semester_text:
        return '여름계절'
    elif '겨울' in semester_text:
        return '겨울계절'

# 삼성 홈페이지에서 다운받은 엑셀에 설정된 주전공명 / 복수전공명 기입
# 부전공까지 있을 경우에는 주석처리된 부분 해제
def determine_major_type(course_type):
    if '복수' in course_type:
        return '복수전공명(이화여대-학사-복수전공)' # 복수전공명 기입
    # elif '부' in course_type:
    #     return '부전공명(이화여대-학사-부전공)'
    else:
        return '주전공명(이화여대-학사-주전공)'  # 주전공명 기입

def convert_grade(grade):
    if grade == 'P' or grade == 'S':
        return 'PASS'
    elif grade == 'F':
        return 'FAIL'
    return grade

# 삼성 홈페이지에서 다운받은 엑셀 업로드
samsung_excel = files.upload()
samsung = list(samsung_excel.keys())[0]

s_wb = openpyxl.load_workbook(samsung)
s_ws = s_wb.active

# 유레카 - 개인성적조회에서 다운받은 엑셀 업로드 (사전 암호 해제 필요)
ewha_excel = files.upload()
ewha = list(ewha_excel.keys())[0]

e_wb = openpyxl.load_workbook(ewha)
e_ws = e_wb.active

start_row = 2

# 교양기타, 전공으로 더 매칭할 항목이 있다면 추가하기
course_type_map = {
    "교양": "교양기타",
    "일반선택": "교양기타",
    "전공기초": "교양기타",
    "전공필수": "전공",
    "전공선택": "전공",
    "복수전공선택": "전공",
    "졸업논문": "전공"
}

course_name_tracker = {}

for row in e_ws.iter_rows(min_row=2, values_only=True):
    seq_num, year, semester, course_code, course_name, course_type, hours, credits, grade, re_year_semester, drop_status = row

    if re_year_semester:
        course_name_tracker[course_name] = start_row
        continue

    major_type = determine_major_type(course_type)
    grade = convert_grade(grade.strip())
    retaken = 'Y' if course_name in course_name_tracker else 'N'

    s_ws[f'A{start_row}'] = start_row - 1  # NO
    s_ws[f'B{start_row}'] = major_type  # 전공명
    s_ws[f'C{start_row}'] = int(year)  # 수강연도
    s_ws[f'D{start_row}'] = determine_semester(semester)  # 학기
    s_ws[f'E{start_row}'] = course_name  # 과목명
    s_ws[f'F{start_row}'] = course_type_map.get(course_type, course_type)  # 과목유형
    s_ws[f'G{start_row}'] = int(credits)  # 취득학점
    s_ws[f'H{start_row}'] = grade  # 성적
    s_ws[f'I{start_row}'] = retaken  # 재수강여부

    start_row += 1

output_excel_file = '이수교과목.xlsx'
s_wb.save(output_excel_file)